In [175]:
from pwn import *
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os
key = os.urandom(16)
guest = b"{'username':'guest_user','role':0}"
def encrypt(key, iv, data):
       cipher = AES.new(key, AES.MODE_CBC, iv)
       return cipher.encrypt(pad(data, 16))

def decrypt(key,token):
      iv = token[:16]
      ct = token[16:]
      cipher = AES.new(key, AES.MODE_CBC, iv)
      dec = cipher.decrypt(ct)
      return dec

#iv = os.urandom(16)
#example = iv + encrypt(key, iv, guest)
r = remote("chal.pctf.competitivecyber.club", 6002)
#r = remote("localhost", 1337)
r.recvuntil(b"Guest: ")
example = bytes.fromhex(r.recvline().strip().decode())
print(f"Example: {example}")

[x] Opening connection to localhost on port 1337
[x] Opening connection to localhost on port 1337: Trying ::1
[x] Opening connection to localhost on port 1337: Trying 127.0.0.1
[+] Opening connection to localhost on port 1337: Done
Example: b'b\xa1\xd7\xf4\x87\x1e\xbd\xd0\xb9\xf8j\xbbP\xd6\xb9\xb0\xdf\n\xe0\xf3<x2\xcf\x1d+=\xf6\x7feV\xfd`L\x88UK=\xeaM\xc4*\x042\xd3c\xfc\xbfb)\x1dMJ\x8fx\xfd7B3\x81>\xe4\xb3\xdc'


In [176]:
example_blocks = [example[i:i+16] for i in range(0, len(example), 16)]
print(f"Example blocks: {len(example_blocks)}")
# iv | b1 | b2 | b3

actual_blocks = [guest[i:i+16] for i in range(0, len(pad(guest,16)), 16)]
print(f"Actual blocks: {len(actual_blocks)}")

target = b"{\"username\":\"administrative_user\",\"role\":1}"
target_blocks = [target[i:i+16] for i in range(0, len(target), 16)]
print(f"Target blocks: {len(target_blocks)}")

deltas = []
for i in range(len(target_blocks)):
    deltas.append(xor(target_blocks[i], actual_blocks[i]))

print(f"Deltas: {len(deltas)}")

Example blocks: 4
Actual blocks: 3
Target blocks: 3
Deltas: 3


In [177]:
def padding_oracle(ciphertext):
    r.sendlineafter(b"): ", ciphertext.hex())
    result = r.recvuntil(b"Access token ")
    if b"Error" in result and b"Input" not in result:
        return False
    return True
#def padding_oracle(ciphertext):
#    dec = decrypt(key, ciphertext)
#    try:
#        unpad(dec, 16)
#        return True
#    except:
#        return False    

In [178]:
def leak_char(data, known, tqdm = __import__('tqdm').tqdm):
    padding_chr = len(known) + 1
    data = data[-32:]
    last_block = data[-16:]
    target_block = data[:-16]
    # set known to padding character
    last_adjust = xor(known, bytes([padding_chr]*len(known)))
    target_block = xor(target_block, b'\x00'*(16-len(last_adjust)) + last_adjust)
    target_idx = 15 - len(known)
    r = range(256)
    for i in tqdm(r):
        query_block = target_block[:target_idx] + bytes([i^target_block[target_idx]]) + target_block[target_idx+1:]
        if padding_oracle(query_block + last_block): # pre we can technically ignore!
            found = padding_chr ^ i
            return bytes([found])
    return None

In [179]:
def leak_block(data, verbose = True):
    known = b""
    while len(known) < 16:
        next = leak_char(data, known)
        if next is None:
            known = known[:-1] + bytes([known[-1] ^ 1])
            continue
        known = next + known
        if verbose:
            print(f"Known: {known}")
    return known

In [180]:

# flashback to when i made the deltas
example_blocks[2] = xor(xor(xor(example_blocks[2], deltas[2])),b'\x00\x00>s>s>s>s>\x19Z\x19\x04T')

# leak second block
b2 = leak_block(example_blocks[1] + example_blocks[2])

# fix b1
example_blocks[1] = xor(b2, target_blocks[1], example_blocks[1])

# leak first block
b1 = leak_block(example_blocks[0] + example_blocks[1])
# fix iv
example_blocks[0] = xor(b1, target_blocks[0], example_blocks[0])


Known: b'\x03'
Known: b'\x9e\x03'
Known: b'o\x9e\x03'
Known: b'Uo\x9e\x03'
Known: b'\x1cUo\x9e\x03'
Known: b'U\x1cUo\x9e\x03'
Known: b'\xa7U\x1cUo\x9e\x03'
Known: b'\x1f\xa7U\x1cUo\x9e\x03'
Known: b'X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\xd4X\x1f\xa7U\x1cUo\x9e\x03'


/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_75784/1442626586.py:2: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendlineafter(b"): ", ciphertext.hex())


Known: b'O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\x00O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'"\x00O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\x14"\x00O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\xe4\x14"\x00O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\x91\xe4\x14"\x00O\xd4X\x1f\xa7U\x1cUo\x9e\x03'
Known: b'\xc9'
Known: b'\xdd\xc9'
Known: b'\xb9\xdd\xc9'
Known: b',\xb9\xdd\xc9'
Known: b'7,\xb9\xdd\xc9'
Known: b'!7,\xb9\xdd\xc9'
Known: b'\xfd!7,\xb9\xdd\xc9'
Known: b'u\xfd!7,\xb9\xdd\xc9'
Known: b'\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'T\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'JT\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'\xd5JT\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'H\xd5JT\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'
Known: b'\x19H\xd5JT\x92\xef\x02u\xfd!7,\xb9\xdd\xc9'


In [181]:
r.sendline(b"".join(example_blocks).hex())
r.recvuntil(" Example: ")
cmd_enc = bytes.fromhex(r.recvline().strip().decode())
differential = xor(pad(b"print(1337)", 16), pad(b"print(flag)", 16))
new_data = xor(cmd_enc, differential + b"\x00"*16)
r.sendline(new_data.hex())
r.recvuntil(b"pctf{")
flag = r.recvuntil(b"}")
print(f"Flag: pctf{{{flag.decode()}}}"[:-1])


Flag: pctf{test}


/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_75784/2860667851.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(b"".join(example_blocks).hex())
/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_75784/2860667851.py:2: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil(" Example: ")
/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_75784/2860667851.py:6: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(new_data.hex())
